In [1]:
import pandas as pd

train = pd.read_csv("train.csv")
#dropping collumns with na rows
train = train.dropna(axis= 1)

#parsing with label encoding for strings
numb = train.select_dtypes(exclude =['object'])
notnumb = train.select_dtypes(exclude =['int64','float64'])
notnumb = notnumb.drop('timestamp', axis=1)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
notnumble = notnumb.apply(le.fit_transform)

date = [None] * 30471
for i in range(30471):
    date[i] = [int(train['timestamp'][i][1:4]),int(train['timestamp'][i][5:7]), int(train['timestamp'][i][8:10])]
df = pd.DataFrame(date,columns=['year','month','day'])

train = numb.join(df)
train = train.join(notnumble)

#train splitting
from sklearn.model_selection import train_test_split

X = train.drop('price_doc', axis=1)
Y = train['price_doc']

X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=0.33, random_state=0)
#learning & results
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
clf = ensemble.GradientBoostingRegressor()
parameters = {'min_samples_split':[2,3,4,5,6],
             'min_samples_leaf':[1,2,3,4,5],
              'max_depth':[1,2,3,4,5,6]}
gridClf = GridSearchCV(clf,parameters,cv=3,verbose=3)
gridClf.fit(X_train,Y_train)

In [3]:
print(gridClf.best_params_)
print(gridClf.best_score_)

{'max_depth': 4, 'min_samples_leaf': 3, 'min_samples_split': 6}
0.6812829943120212


In [4]:
from sklearn.metrics import mean_squared_log_error
print(mean_squared_log_error(Y_test,gridClf.predict(X_test)))

0.22416279087363777
